In [3]:
import pandas as pd
import math
import numpy as np

In [ ]:
# rating = pd.read_csv('ydata-ymovies-user-movie-ratings-train-v1_0.txt', sep='\t',names=['user_id', 'movie_id', 'rating(1-13)','rating(1-5)'], encoding='latin-1');
# movie_content = pd.read_csv('movie_db_yoda', sep='[\t]',engine='python' ,names=['movie_id','title','synopsis','run_time','MPAA_rating','reason_for_MPAA_rating','release_date','distributor','poster_URL','genres','directors','director_ids','crew_members','crew_ids','types_of_crew','actors','actor_ids','critic_rating','number_of_critic_ratings','number_of_awards_won','number_of_awards_nominated','list_of_awards_won','list_of_awards_nominated','Movie_Mom_rating','Movie_Mom_review','review_summaries','review_owners','captions','url_greg_preview','url_dvd_review','GNPP','average_rating','rating_users'], encoding='latin-1')
# rating
# movie_content
# shashanksi2009@
# print(movie_content.iloc[[62]])
# len(['movie_id','title','synopsis','run_time','MPAA_rating','reason_for_MPAA_rating','release_date','distributor','poster_URL','genres','directors','director_ids','crew_members','crew_ids','types_of_crew','actors','actor_ids','critic_rating','number_of_critic_ratings','number_of_awards_won','number_of_awards_nominated','list_of_awards_won','list_of_awards_nominated','Movie_Mom_rating','Movie_Mom_review','review_summaries','review_owners','captions','url_greg_preview','url_dvd_review','GNPP','average_rating','rating_users'])

In [4]:
# getting the ratings data in pandas dataframe
ratings = pd.read_csv('../Yahoo movies/ydata-ymovies-user-movie-ratings-train-v1_0.txt', sep='\t',names=['user_id', 'movie_id', 'rating_13', 'rating_5'])
num_users=ratings['user_id'].value_counts().size # number of users
# ratings['user_id'].drop_duplicates().count()
num_items=ratings['movie_id'].value_counts().size # number of items
print('users - ',num_users)
print('items - ',num_items)
print('ratings - ',ratings.shape[0])  # number of  items
# average number of ratings per-user
ratings.groupby(['user_id'])['rating_13'].count().sum()/num_users
#average number of ratings per-item
ratings.groupby(['movie_id'])['rating_13'].count().sum()/num_items
# minimum number of movies rated by any user
min(ratings.groupby(['user_id'])['rating_13'].count())
# ratings


users -  7642
items -  11916
ratings -  211231


10

## Popularity Model

In [5]:
# computing the most popular movies
# This is not actually needed for below function to work
movie_popularity_df = ratings.groupby('movie_id')['rating_5'].sum().sort_values(ascending=False).reset_index()
# movie_popularity_df

In [103]:
# cell for rough workuser1]
# ratings.groupby('user_id')['movie_id'].count()

In [6]:
class PopularityRecommender:
    
    # constructor
    def __init__(self, popularity_df, movies_df=None):
        self.popularity_df = popularity_df
        self.movies_df = movies_df
    
    # function to recommend items
    def recommend_items(self, user_id, movies_to_ignore=[], topn=10):
        recommended_df = self.popularity_df[~self.popularity_df['movie_id'].isin(movies_to_ignore)]\
        .sort_values('rating_5',ascending=False)\
        .head(topn)
        return recommended_df
    

In [7]:
popularity_model = PopularityRecommender(movie_popularity_df)
user_id = 1
# print(ratings.shape)
# movies rated by the user i.e. movies to ignore
rated_movies = ratings[ratings['user_id']==user_id].movie_id
recommended_df = popularity_model.recommend_items(user_id, rated_movies, 5)
recommended_df

,movie_id,rating_5
1,1808405428,14092
3,1807428853,11862
6,1807432594,8940
7,1808403030,8241
8,1807592188,7603


## Collaborative Filtering Model (user-user)

In [8]:
# compute similarity between two users based on euclidean distance
def euclidean_similarity(user1, user2):
    # find all the movies that are rated by both the users
    user1_movies = ratings[ratings['user_id']==user1]
    user2_movies = ratings[ratings['user_id']==user2]
    similar_movies_df = pd.merge(user1_movies, user2_movies, how='inner', on=['movie_id'])
    similar_movies_df = similar_movies_df[['movie_id', 'rating_5_x','rating_5_y']]
#     print(similar_movies_df)
    temp_df = (similar_movies_df['rating_5_x'] - similar_movies_df['rating_5_y'])**2
    return 1/(1+sum(temp_df))

In [9]:
# to compute similarity between two users based on pearson correlation formula
# it is not used in the implementation below
# python has an inbuilt function corr() for finding correlations between two vectors
def pearson_similarity(user1, user2):
    # find all movies that are rated by both the users
    user1_movies = ratings[ratings['user_id']==user1]
    user2_movies = ratings[ratings['user_id']==user2]
    similar_movies_df = pd.merge(user1_movies, user2_movies, how='inner', on=['movie_id'])
    similar_movies_df = similar_movies_df[['movie_id', 'rating_5_x','rating_5_y']]
    
    if(similar_movies_df.shape[0]==0): # no similar movie between the two users
        return 0
#     print(similar_movies_df)
    avg1 = sum(similar_movies_df['rating_5_x'])/similar_movies_df.shape[0]
    avg2 = sum(similar_movies_df['rating_5_y'])/similar_movies_df.shape[0]
    # apply the pearson correlation formula
    temp1 = (similar_movies_df['rating_5_x']-avg1)
    temp2 = (similar_movies_df['rating_5_y']-avg2)
    num = sum(temp1.multiply(temp2))
    den = (sum(temp1**2))**0.5 * (sum(temp2**2))**0.5
    return num/den
    
# pearson_similarity(6,2)

In [27]:
# for calculating average rating of a user
def my_average(user): 
    temp = ratings[ratings['user_id']==user].rating_5
    avg = temp.sum()/temp.count()
    return avg

# for returning a vector containing all ratings of a user
# this is wrong
def get_correlation(user1, user2):
    user1_data = ratings[ratings['user_id']==user1]
    user2_data = ratings[ratings['user_id']==user2]

    
    s1 = pd.merge(user1_data, user2_data, how='inner', on=['movie_id']) # getting the data for movies rated by both user
    print(s1)
#     print(s1.rating_5_x.isnull())
#     print(s1.rating_5_y.isnull())
    t1 = s1.rating_5_x
    t2 = s1.rating_5_y
#     print(t1, t2)
#     s1.fillna(np.nan, inplace=True)
    pearson_correlation = t1.corr(t2,  method='pearson')
#     print(s1.rating_5_x)
#     print(s1.rating_5_y)
#     print(s1.rating_5_x)
#     print("correlation = ",pearson_correlation)
    temp_data = pd.DataFrame({'A':[5,5], 'B':[5,1]})
    print(np.corrcoef(temp_data.A, temp_data.B))
    return pearson_correlation
    
get_correlation(5,10)
# ratings.loc[(ratings['user_id']==8) & (ratings['movie_id']==1800019594)].rating_5

   user_id_x    movie_id  rating_13_x  rating_5_x  user_id_y  rating_13_y  \
0          5  1808440470           12           5         10           13   
1          5  1808412006           12           5         10            1   

   rating_5_y  
0           5  
1           1  
[[nan nan]
 [nan  1.]]


/home/garvit/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3183: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/garvit/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3184: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


nan

In [98]:
# For computing correlation between every pair of user
# computationally very expensive  :-(
# not using
user_correlation = dict()
for user1 in ratings['user_id'].drop_duplicates():
    print("started", user1)
    for user2 in ratings['user_id'].drop_duplicates():
        user1_data = ratings[ratings['user_id']==user1]
        user2_data = ratings[ratings['user_id']==user2]
#         print(user1, user2)
#         s1 = pd.merge(user1_data, user2_data, how='outer', on=['movie_id']) # getting the data for movies rated by both user
#         s1.fillna(0, inplace=True)
        # code for merging user1 and user2 data
        s1 = pd.DataFrame(columns=['user1', 'user2'])
        for movie in user1_data['movie_id']:
            temp=user2_data[user2_data['movie_id']==movie].rating_5
            temp1 = user1_data[user1_data['movie_id']==movie].rating_5
            if(len(temp.index) != 0):
                s1 = s1.append({'user1': temp1, 'user2': temp}, ignore_index=True)
            else:
                s1 = s1.append({'user1': temp1, 'user2': 0}, ignore_index=True)
#         pearson_correlation = s1.rating_5_x.corr(s1.rating_5_y)
#         print(s1.columns)
        pearson_correlation = s1.user1.astype(float).corr(s1.user2.astype(float))    
        user_correlation[(user1,user2)] = pearson_correlation
print(user_correlation)

In [11]:
# Rough work
user1_data = ratings[ratings['user_id']==5]
user2_data = ratings[ratings['user_id']==8]
# print(user1_data)
s1 = pd.merge(user1_data, user2_data, how='outer', on=['movie_id']) # getting the data for movies rated by both user
s1.fillna(0, inplace=True)
pearson_correlation = s1.rating_5_x.corr(s1.rating_5_y)
pearson_correlation
user1_data.rating_5.corr(user2_data.rating_5)

nan

In [12]:
# function to predict rating by user to movie
def predict(user, movie): 
    # calculate the predicted score of movie
    numerator = 0
    denominator = 0
    cnt = 0
    for user2 in ratings['user_id'].drop_duplicates():
        rate_v_i_series = ratings.loc[(ratings['user_id']==user2) & (ratings['movie_id']==movie)].rating_5 # rating given by user2 to movie
        if(rate_v_i_series.empty): # user has not rated the movie
            continue
        cnt += 1
        rate_v_i = rate_v_i_series.iloc[0]
        avg_v = my_average(user2) # average rating given by user2
        weight = get_correlation(user, user2) # Pearson correlation between user and user2
        numerator += (rate_v_i - avg_v)*weight
        denominator += weight
    rating_i = numerator/denominator + my_average(user) # Predicted rating of movie i
    return rating_i        

In [13]:
# function for finding root mean square error
def find_error(test):
    error = 0
    for index, row in test.iterrows():
        value = predict(row['user_id'], row['movie_id'])
        error += (value - row['rating_5'])**2
    # Mean square error
    error = error/test.shape[0]
    # root mean square error
    return math.sqrt(error)

In [14]:
# import test data to pandas dataframe
test = pd.read_csv('../Yahoo movies/ydata-ymovies-user-movie-ratings-test-v1_0.txt', sep='\t', names = ['user_id', 'movie_id', 'rating_13', 'rating_5'])
test.shape

(10136, 4)

In [15]:
print("RMSE ---> ", find_error(test.head(1))) # evaluating the model

RMSE --->  0.5898688970879378
